<a href="https://colab.research.google.com/github/danilovabg/ML-DL/blob/master/detection_efficientnet_ssd.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install mmcv-full

In [ ]:
# Install mmdetection
!rm -rf mmdetection
!git clone https://github.com/open-mmlab/mmdetection.git
%cd mmdetection

!pip install -e .


In [17]:
import numpy as np
import torch
import torch.nn.functional as F
import mmcv


In [18]:
import torch, torchvision
print(torch.__version__, torch.cuda.is_available())
import mmdet
print(mmdet.__version__)

# Check mmcv installation
from mmcv.ops import get_compiling_cuda_version, get_compiler_version
print(get_compiling_cuda_version())
print(get_compiler_version())

1.13.1+cu116 True
2.27.0
11.2
GCC 9.4


In [3]:
from google.colab import drive
drive.mount('/content/gdrive/')

Drive already mounted at /content/gdrive/; to attempt to forcibly remount, call drive.mount("/content/gdrive/", force_remount=True).


In [19]:
from mmdet.models import SingleStageDetector
from mmdet.apis import set_random_seed
from mmdet.apis import init_detector, inference_detector
from mmcv import Config

In [29]:
cfg = Config.fromfile('/content/mmdetection/configs/ssd/ssd300_coco.py')

cfg.model['backbone'] = dict(
        # _delete_=True, # Delete the backbone field in _base_
        type='EfficientNet', # Using MobileNetV3 from mmcls
        arch='b3',# Modify out_indices
        
        drop_path_rate=0.0, out_indices=(3,), #frozen_stages=3, 
        conv_cfg={'type': 'Conv2dAdaptivePadding'}, 
        norm_cfg={'eps': 0.001, 'type': 'BN'}, 
        act_cfg={'type': 'Swish'}, norm_eval=False, with_cp=False, 
        init_cfg=[{'type': 'Kaiming', 'layer': 'Conv2d'}, {'type': 'Constant', 'layer': ['_BatchNorm', 'GroupNorm'], 'val': 1},])

cfg.data_root = '/content/gdrive/MyDrive/mmdetection2/data/coco'
cfg.data.train.dataset.data_root = '/content/gdrive/MyDrive/mmdetection2/'
cfg.data.train.dataset.ann_file = '/content/gdrive/MyDrive/mmdetection2/data/coco/annotations/instances_train2017.json'
cfg.data.val.ann_file = '/content/gdrive/MyDrive/mmdetection2/data/coco/annotations/instances_val2017.json'

# Set up working dir to save files and logs.
cfg.work_dir = '/content/gdrive/MyDrive/mmdetection2/coco'
# The original learning rate (LR) is set for 8-GPU training.
# We divide it by 8 since we only use one GPU.
# ssd300_coco and ssd300_voc with 4gpu, 24 imgs per gpu and lr of 3e-3
cfg.optimizer.lr = 0.02/ 8.  
# cfg.lr_config.warmup = None
cfg.lr_config.policy = 'step'
cfg.lr_config.warmup = 'linear'
cfg.lr_config.warmup_ratio = 1.0 / 1e10
        # cfg.runner = dict(type='EpochBasedRunner', max_epochs=10) 
cfg.log_config.interval = 10

# Change the evaluation metric since we use customized dataset.
cfg.evaluation.metric = 'mAP'
# We can set the evaluation interval to reduce the evaluation times
cfg.evaluation.interval = 12
# We can set the checkpoint saving interval to reduce the storage cost
cfg.checkpoint_config.interval = 12


# Set seed thus the results are more reproducible
cfg.seed = 4
set_random_seed(4, deterministic=False)
cfg['device'] = 'cuda'
cfg.gpu_ids = range(1)
# cfg['device'] = 'cpu'
# We can initialize the logger for training and have a look
# at the final config used for training

print(f'Config:\n{cfg.pretty_text}')

Config:
input_size = 300
model = dict(
    type='SingleStageDetector',
    backbone=dict(
        type='EfficientNet',
        arch='b3',
        drop_path_rate=0.0,
        out_indices=(3, ),
        conv_cfg=dict(type='Conv2dAdaptivePadding'),
        norm_cfg=dict(eps=0.001, type='BN'),
        act_cfg=dict(type='Swish'),
        norm_eval=False,
        with_cp=False,
        init_cfg=[
            dict(type='Kaiming', layer='Conv2d'),
            dict(type='Constant', layer=['_BatchNorm', 'GroupNorm'], val=1)
        ]),
    neck=dict(
        type='SSDNeck',
        in_channels=(512, 1024),
        out_channels=(512, 1024, 512, 256, 256, 256),
        level_strides=(2, 2, 1, 1),
        level_paddings=(1, 1, 0, 0),
        l2_norm_scale=20),
    bbox_head=dict(
        type='SSDHead',
        in_channels=(512, 1024, 512, 256, 256, 256),
        num_classes=80,
        anchor_generator=dict(
            type='SSDAnchorGenerator',
            scale_major=False,
            input_si

In [33]:
from mmdet.datasets import build_dataset
from mmdet.models import build_detector
from mmdet.apis import train_detector

device = 'cuda' if torch.cuda.is_available() else 'cpu'

model_detector = build_detector( cfg.model, train_cfg=cfg.get('train_cfg'), test_cfg=cfg.get('test_cfg'))
# mmcv.mkdir_or_exist(os.abspath(cfg.work_dir))
datasets = [build_dataset(cfg.data.train)]

train_detector(model_detector, datasets, cfg, distributed=False, validate=True)

loading annotations into memory...
Done (t=24.58s)
creating index...
index created!


/usr/local/lib/python3.8/dist-packages/torch/utils/data/dataloader.py:554: UserWarning: This DataLoader will create 3 worker processes in total. Our suggested max number of worker in current system is 2, which is smaller than what this DataLoader is going to create. Please be aware that excessive worker creation might get DataLoader running slow or even freeze, lower the worker number to avoid potential slowness/freeze if necessary.
  warnings.warn(_create_warning_msg(
2023-01-25 13:56:41,696 - mmdet - INFO - Automatic scaling of learning rate (LR) has been disabled.


loading annotations into memory...


2023-01-25 13:56:44,778 - mmdet - INFO - Start running, host: root@f6eaeabeda93, work_dir: /content/gdrive/MyDrive/mmdetection2/coco
2023-01-25 13:56:44,780 - mmdet - INFO - Hooks will be executed in the following order:
before_run:
(VERY_HIGH   ) StepLrUpdaterHook                  
(NORMAL      ) CheckpointHook                     
(LOW         ) EvalHook                           
(VERY_LOW    ) TextLoggerHook                     
 -------------------- 
before_train_epoch:
(VERY_HIGH   ) StepLrUpdaterHook                  
(NORMAL      ) NumClassCheckHook                  
(LOW         ) IterTimerHook                      
(LOW         ) EvalHook                           
(VERY_LOW    ) TextLoggerHook                     
 -------------------- 
before_train_iter:
(VERY_HIGH   ) StepLrUpdaterHook                  
(LOW         ) IterTimerHook                      
(LOW         ) EvalHook                           
 -------------------- 
after_train_iter:
(ABOVE_NORMAL) OptimizerHook 

Done (t=2.99s)
creating index...
index created!
train


RuntimeError: ignored

In [ ]:
import torch, torchvision
print(torch.__version__, torch.cuda.is_available())